In [1]:
import json
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas.io.sql as psql
import psycopg2 as pg
import seaborn as sns
import torch
import torch.nn as nn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm, trange

%matplotlib inline
connection = pg.connect("host=localhost dbname=MIMIC-III user=postgres password=1027")


In [2]:
queries = json.load(open("queries.json"))


In [3]:
patients = psql.read_sql('SELECT subject_id, hadm_id\nFROM diagnoses_icd', connection)
patients

,subject_id,hadm_id
0,67819,134703
1,67819,134703
2,67819,134703
3,67819,134703
4,67819,134703
...,...,...
651042,67814,126376
651043,67814,126376
651044,67814,126376
651045,67814,126376


In [4]:
def uniq_merge(left, right, on="hadm_id", drop_patient_id=True):
    if drop_patient_id and 'subject_id' in right.columns:
        right = right.drop(columns=['subject_id'])
    df = left.merge(right, on=on, how="left")
    df = df.sort_values(by=df.columns.tolist())

    merged_df = df.groupby(on).last().reset_index()
    return merged_df

patients = uniq_merge(patients, psql.read_sql("SELECT subject_id, gender FROM patients", connection), on="subject_id", drop_patient_id=False)

df = patients.copy()
for query in tqdm(list(queries.keys())[1:]):
    print(query)
    ret = psql.read_sql(queries[query], connection)
    df = uniq_merge(df, ret)
    print(df.columns.tolist())
    # ret.to_csv(f"./queries/{query}.csv", index=False)
    # ret.set_index("subject_id", inplace=True)
    # patients = patients.join(ret, how="inner")
# patients.head(5)
ret = psql.read_sql("SELECT subject_id, hadm_id, admittime, deathtime FROM admissions", connection)
df = uniq_merge(df, ret)
uniq = df.copy()
df


  0%|          | 0/30 [00:00<?, ?it/s]

age
['hadm_id', 'subject_id', 'gender', 'age']
admission_marital
['hadm_id', 'subject_id', 'gender', 'age', 'admission_type', 'marital_status']
icu_department_icusty_id
['hadm_id', 'subject_id', 'gender', 'age', 'admission_type', 'marital_status', 'icustay_id']
comorbidities
['hadm_id', 'subject_id', 'gender', 'age', 'admission_type', 'marital_status', 'icustay_id', 'icd9_code']
systolic_bp
['hadm_id', 'subject_id', 'gender', 'age', 'admission_type', 'marital_status', 'icustay_id', 'icd9_code', 'systolic']
diastolic_bp
['hadm_id', 'subject_id', 'gender', 'age', 'admission_type', 'marital_status', 'icustay_id', 'icd9_code', 'systolic', 'diastolic']
heart_rate
['hadm_id', 'subject_id', 'gender', 'age', 'admission_type', 'marital_status', 'icustay_id', 'icd9_code', 'systolic', 'diastolic', 'heart rate']
respiratory_rate
['hadm_id', 'subject_id', 'gender', 'age', 'admission_type', 'marital_status', 'icustay_id', 'icd9_code', 'systolic', 'diastolic', 'heart rate', 'respiratory rate']
white_

,hadm_id,subject_id,gender,age,admission_type,marital_status,icustay_id,icd9_code,systolic,diastolic,...,asparate aminotransferase,total bilirubin,urea nitrogen,creatinine,lactate,calcium,anion gap,los,admittime,deathtime
0,100007,23018,F,"26942 days, 5:33:00",EMERGENCY,MARRIED,217937.0,None,133.954545,57.590909,...,NaN,NaN,13.000000,0.65,NaN,8.700000,11.000000,4.0998,2145-03-31 05:33:00,NaT
1,100009,533,M,"22187 days, 15:56:00",EMERGENCY,MARRIED,253656.0,None,80.608696,39.347826,...,55.0,0.3,19.000000,0.80,1.100,NaN,13.000000,2.4908,2162-05-16 15:56:00,NaT
2,100010,55853,F,"19897 days, 7:15:00",ELECTIVE,MARRIED,271147.0,None,100.979167,54.520833,...,NaN,NaN,11.000000,1.05,0.825,8.600000,9.500000,1.5940,2109-12-10 07:15:00,NaT
3,100011,87977,M,"7847 days, 4:51:00",EMERGENCY,SINGLE,214619.0,None,145.423077,73.038462,...,NaN,NaN,11.666667,1.20,2.300,7.366667,9.666667,11.5029,2177-08-29 04:51:00,NaT
4,100012,60039,M,"24711 days, 11:48:00",EMERGENCY,MARRIED,239289.0,None,NaN,NaN,...,24.0,2.0,14.000000,1.00,NaN,NaN,14.000000,3.5763,2177-03-12 11:48:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46515,199993,20785,M,"22087 days, 18:01:00",EMERGENCY,DIVORCED,271752.0,None,61.696970,55.440000,...,36.0,0.8,10.500000,0.90,NaN,8.350000,14.000000,24.5895,2161-10-23 18:01:00,NaT
46516,199994,23761,F,"21274 days, 18:47:00",EMERGENCY,SINGLE,296723.0,None,107.000000,51.482759,...,22.0,0.3,26.000000,3.40,0.750,9.100000,17.000000,6.9597,2188-07-07 18:47:00,NaT
46517,199995,19412,M,"8621 days, 17:35:00",EMERGENCY,SINGLE,255092.0,V1259,NaN,NaN,...,14.0,0.1,15.000000,0.70,NaN,NaN,15.000000,2.8532,2137-12-11 17:35:00,NaT
46518,199998,27200,M,"25243 days, 16:26:00",EMERGENCY,MARRIED,293589.0,None,NaN,NaN,...,30.0,0.7,16.000000,0.90,NaN,10.200000,15.000000,1.0623,2119-02-18 16:26:00,NaT


In [5]:
df['age'] = df['age'].apply(lambda x: 0.0 if len(str(x).split()[0])>5 else pd.Timedelta(x) / pd.Timedelta('365.25 days'))
df.drop(df[df["age"] < 18].index, inplace=True)

In [6]:
df['delta_deathtime'] = df['deathtime'] - df['admittime']
df['mortality_30'] = df['delta_deathtime'].apply(lambda x: 1 if x < pd.Timedelta('30 days') else 0)
df['mortality_60'] = df['delta_deathtime'].apply(lambda x: 1 if x < pd.Timedelta('60 days') else 0)
df['mortality_90'] = df['delta_deathtime'].apply(lambda x: 1 if x < pd.Timedelta('90 days') else 0)

In [7]:
df.columns.tolist()

['hadm_id',
 'subject_id',
 'gender',
 'age',
 'admission_type',
 'marital_status',
 'icustay_id',
 'icd9_code',
 'systolic',
 'diastolic',
 'heart rate',
 'respiratory rate',
 'wbc',
 'neutrophil',
 'lymphocytes',
 'sodium',
 'chloride',
 'platelet',
 'rcv',
 'mcv',
 'hematocrit',
 'glucose',
 'prothrombin time',
 'partial prothrombin time',
 'albumin',
 'alanine aminotransferase',
 'asparate aminotransferase',
 'total bilirubin',
 'urea nitrogen',
 'creatinine',
 'lactate',
 'calcium',
 'anion gap',
 'los',
 'admittime',
 'deathtime',
 'delta_deathtime',
 'mortality_30',
 'mortality_60',
 'mortality_90']

In [8]:
print(df['gender'].unique())
print(df['admission_type'].unique())
print(df['marital_status'].unique())

['F' 'M']
['EMERGENCY' 'ELECTIVE' 'URGENT']
['MARRIED' 'SINGLE' 'WIDOWED' 'DIVORCED' None 'UNKNOWN (DEFAULT)'
 'SEPARATED' 'LIFE PARTNER']


In [9]:
df['gender'] = df['gender'].replace(['M', 'F'], [0, 1])
df['admission_type'] = df['admission_type'].replace(['ELECTIVE', 'EMERGENCY', 'URGENT'], [0, 1, 2])
df['marital_status'] = df['marital_status'].replace([pd.NaT, np.nan, None, 'UNKNOWN (DEFAULT)', 'SINGLE', 'MARRIED', 'DIVORCED', 'SEPARATED', 'WIDOWED'], [0, 0, 0, 0, 1, 2, 3, 4, 5])

In [10]:
df.to_csv("patients.csv", index=False)